# Projet Quadratic Assignment Problem

In [32]:
import pandas as pd

from Models.TaillardParser import TaillardParser
from Services.Fitness import Fitness
from Services.Voisinage import Voisinage
from Services.Recuit import Recuit
from Services.Tabou import Tabou

parser = TaillardParser('tai12a.dat')

distances = parser.get_distance_matrix()
connexions = parser.get_connexion_matrix()

pd.DataFrame(distances)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,27,85,2,1,15,11,35,11,20,21,61
1,27,0,80,58,21,76,72,44,85,94,90,51
2,85,80,0,3,48,29,90,66,41,15,83,96
3,2,58,3,0,74,45,65,40,54,83,14,71
4,1,21,48,74,0,77,36,53,37,26,87,76
5,15,76,29,45,77,0,91,13,29,11,77,32
6,11,72,90,65,36,91,0,87,67,94,79,2
7,35,44,66,40,53,13,87,0,10,99,56,70
8,11,85,41,54,37,29,67,10,0,99,60,4
9,20,94,15,83,26,11,94,99,99,0,56,2


In [33]:
pd.DataFrame(connexions)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,21,95,82,56,41,6,25,10,4,63,6.0
1,21,0,44,40,75,79,0,89,35,9,1,85.0
2,95,44,0,84,12,0,26,91,11,35,82,26.0
3,82,40,84,0,69,56,86,45,91,59,18,76.0
4,56,75,12,69,0,39,18,57,36,61,36,21.0
5,41,79,0,56,39,0,71,11,29,82,82,6.0
6,6,0,26,86,18,71,0,71,8,77,74,30.0
7,25,89,91,45,57,11,71,0,89,76,76,40.0
8,10,35,11,91,36,29,8,89,0,93,56,1.0
9,4,9,35,59,61,82,77,76,93,0,50,4.0


In [46]:
voisinage = Voisinage(distances)
x = voisinage.get_random_x()
fitness = Fitness(connexions, distances)
tabou = Tabou(distances, connexions, voisinage, fitness)
print(x)

[11, 8, 1, 2, 7, 10, 3, 5, 9, 4, 6, 0]


In [74]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

# out = interactive_output(voisinage.get_random_x)

interact_manual(tabou.resolve, x=fixed(voisinage.get_random_x()), tabousize=range(0,10), maxIter=range(0,1000,100))

print("solution optimale: ")
print([7,0,5,1,10,9,2,4,8,6,11,3])
print("Fitness optimale: ")
print(fitness.calcul([7,0,5,1,10,9,2,4,8,6,11,3]))

interactive(children=(Dropdown(description='tabousize', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), Drop…

solution optimale: 
[7, 0, 5, 1, 10, 9, 2, 4, 8, 6, 11, 3]
Fitness optimale: 
112208


In [69]:
a = widgets.IntSlider()
b = widgets.IntSlider()
c = widgets.IntSlider()
ui = widgets.HBox([a, b, c])
def f(a, b, c):
    print((a, b, c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)

Output()